In [ ]:
# Read json files containing Pinterest data into DataFrames
df_pin = spark.read.json("/mnt/0a1667ad2f7f/topics/0a1667ad2f7f.pin/partition=0/")
df_geo = spark.read.json("/mnt/0a1667ad2f7f/topics/0a1667ad2f7f.geo/partition=0/")
df_user = spark.read.json("/mnt/0a1667ad2f7f/topics/0a1667ad2f7f.user/partition=0/")

In [ ]:
df_pin.printSchema()
df_geo.printSchema()
df_user.printSchema()

In [ ]:
# Task 1: Clean the df_pin DataFrame

# Replace empty entries and entries that do not contain relevant data in each column with None
cleaned_df_pin = (df_pin.replace({'No description available Story format': None}, subset=['description'])
                    .replace({'No description available': None}, subset=['description'])
                    .replace({'User Info Error': None}, subset=['follower_count'])
                    .replace({'Image src Error.': None}, subset=['image_src'])
                    .replace({'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e': None}, subset=['tag_list'])
                    .replace({'No Title Data Available': None}, subset=['title'])
                    .replace({'User Info Error': None}, subset=['poster_name']))

# Perform necessary transformations on the follower_count to ensure every entry is a number.
from pyspark.sql.functions import regexp_replace
cleaned_df_pin = cleaned_df_pin.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
cleaned_df_pin = cleaned_df_pin.withColumn("follower_count", regexp_replace("follower_count", "M", "000000"))
# Make sure the data type of this column is an integer
cleaned_df_pin = cleaned_df_pin.withColumn("follower_count", cleaned_df_pin["follower_count"].cast("integer"))

# Ensure each column containing numeric data has a numeric data type
cleaned_df_pin = cleaned_df_pin.withColumn("downloaded", cleaned_df_pin["downloaded"].cast("integer"))
cleaned_df_pin = cleaned_df_pin.withColumn("index", cleaned_df_pin["index"].cast("integer"))

# Clean the data in the save_location column to include only the save location path
cleaned_df_pin = cleaned_df_pin.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))

# Rename the index column to ind
cleaned_df_pin = cleaned_df_pin.withColumnRenamed("index", "ind")

# Reorder the DataFrame columns
cleaned_df_pin = cleaned_df_pin.select("ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category")
# This removes the 'download' column

In [ ]:
# Task 2: Clean df_geo DataFrame

# Create new column 'coordinates' containing an array based on the 'latitude' and 'longitude' columns
from pyspark.sql.functions import array
cleaned_df_geo = df_geo.withColumn("coordinates", array("latitude", "longitude"))

# Drop the latitude and longitude columns from DataFrame
cleaned_df_geo = (cleaned_df_geo.drop("latitude")
                                .drop("longitude"))

# Convert the timestamp column from a string to a timestamp data type
from pyspark.sql.functions import to_timestamp
cleaned_df_geo = cleaned_df_geo.withColumn("timestamp", to_timestamp("timestamp"))

# Reorder the DataFrame columns
cleaned_df_geo = cleaned_df_geo.select("ind", "country", "coordinates", "timestamp")

In [ ]:
# Task 3: Clean df_user DataFrame

# Create new column user_name that concatenates the first_name and last_name columns
from pyspark.sql.functions import concat
cleaned_df_user = df_user.withColumn("user_name", concat("first_name", "last_name"))

# Drop the first_name and last_name columns from the DataFrame
cleaned_df_user = (cleaned_df_user.drop("first_name")
                                    .drop("last_name"))

# Convert the date_joined column from a string to a timestamp data type
from pyspark.sql.functions import to_timestamp
cleaned_df_user = cleaned_df_user.withColumn("date_joined", to_timestamp("date_joined"))

# Reorder the DataFrame columns
cleaned_df_user = cleaned_df_user.select("ind", "user_name", "age", "date_joined")

In [ ]:
display(cleaned_df_pin)
display(cleaned_df_geo)
display(cleaned_df_user)

In [ ]:
cleaned_df_pin.printSchema()
cleaned_df_geo.printSchema()
cleaned_df_user.printSchema()

In [ ]:
# Task 4: Find the most popular category in each country

# Join pin and geo DataFrames
combined_df_t4 = cleaned_df_pin.join(cleaned_df_geo, cleaned_df_pin["ind"] == cleaned_df_geo["ind"], how="inner")

# Identify most popular category in each country
from pyspark.sql.functions import col, count, rank
from pyspark.sql.window import Window

# # Use groupBy and aggregation functions
# grouped_df_t4 = combined_df_t4.groupBy(["country", "category"]).agg(count("category").alias("category_count"))
# grouped_df_t4 = grouped_df_t4.orderBy(["country", "category_count"], ascending=[True, False])
# # This does not quite work - you can't filter to keep just the top category

# Using window function to present only the most popular category

grouped_df_t4 = (combined_df_t4.groupBy(["country", "category"])
                                .agg(count("category").alias("category_count")))
# Create Window specification
window_spec = Window.partitionBy("country").orderBy(col("category_count").desc())
# Apply window function
grouped_df_t4 = grouped_df_t4.withColumn("rank", rank().over(window_spec))
# Filter to keep only top category in each country
grouped_df_t4 = grouped_df_t4.filter(col("rank") == 1).drop("rank")

In [ ]:
display(grouped_df_t4)

In [ ]:
# Task 5: Find which was the most popular category each year

# Join pin and geo DataFrames
combined_df_t5 = cleaned_df_pin.join(cleaned_df_geo, cleaned_df_pin["ind"] == cleaned_df_geo["ind"], how="inner")

# Identify most popular category each year
from pyspark.sql.functions import year, col, count, rank
from pyspark.sql.window import Window

# Create post_year column
combined_df_t5 = combined_df_t5.withColumn("post_year", year("timestamp"))

# Filter to restrict post_year to between 2018 and 2022
combined_df_t5 = combined_df_t5.filter((col("post_year") >= 2018) & (col("post_year") <= 2022))

# # Use groupBy and aggregation functions
grouped_df_t5 = (combined_df_t5.groupBy(["post_year", "category"])
                            .agg(count("category").alias("category_count")))

# Create a Window specification
window_spec = Window.partitionBy("post_year").orderBy(col("category_count").desc())

# Apply window function to rank yearly category posts
grouped_df_t5 = grouped_df_t5.withColumn("rank", rank().over(window_spec))

# Filter to keep only top category in each year
grouped_df_t5 = grouped_df_t5.filter(col("rank") == 1).drop("rank")

In [ ]:
display(grouped_df_t5)

In [ ]:
# Task 6: Find the user with most followers in each country

# Step 1: For each country find the user with the most followers
from pyspark.sql.functions import max, desc
# Join cleaned pin and geo DataFrames
combined_df_t6 = cleaned_df_pin.alias("pin").join(cleaned_df_geo.alias("geo"), cleaned_df_pin["ind"] == cleaned_df_geo["ind"], how="inner")

# Aggregation to find maximum follower count per country
max_follower_per_country = combined_df_t6.groupBy(["geo.country"]).agg(max("pin.follower_count").alias("max_follower_count"))

# Rename country_column in max_follower_per_country to avoid naming conflict later
max_follower_per_country = max_follower_per_country.withColumnRenamed("country", "country_agg")

# Join max_follower_per_country DataFrame with combined_df_t6 to get poster_name for each maximum follower count
df_with_max_follower = combined_df_t6.join(max_follower_per_country,
                                           (combined_df_t6["geo.country"] == max_follower_per_country["country_agg"]) & (combined_df_t6["pin.follower_count"] == max_follower_per_country["max_follower_count"]), how="inner")

# Select desired columns
result_df_step1 = df_with_max_follower.select("country", "poster_name", "follower_count").distinct()

# Step 2: Based on the above query, find the country with the user with most followers
# Return a DataFrame with the following columns: country, follower_count
grouped_df_step2 = result_df_step1.groupBy("country").agg(max("follower_count").alias("follower_count"))

# Order grouped_df_step2 by follower_count
ordered_df_step2 = grouped_df_step2.orderBy(desc("follower_count"))

# Select country with highest follower_count
result_df_step2 = ordered_df_step2.select(["country", "follower_count"]).limit(1)

In [ ]:
display(result_df_step2)

In [ ]:
# Task 7: Find the most popular category for different age groups
#  18-24, 25-35, 36-50, +50
# Query should return a DataFrame containing the following columns;
# age_group, category, category_count

from pyspark.sql.functions import when
# Join pin and user DataFrames
combined_df_t7 = cleaned_df_pin.join(cleaned_df_user, cleaned_df_pin["ind"] == cleaned_df_user["ind"], "inner")
# Select relevant columns
combined_df_t7 = combined_df_t7.select("age", "category")
# Categorise age into ranges
combined_df_t7 = combined_df_t7.withColumn("age_group",
                                           when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                           .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                           .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                           .otherwise("+50"))

# Group by age_group and count the categories
grouped_df_t7 = combined_df_t7.groupBy(["age_group", "category"]).agg(count("category").alias("category_count"))

# Find the most popular category within each age group
# Rank and filter each age group

# Create window specification that partitions data by age_group and orders by category_count
window_spec = Window.partitionBy("age_group").orderBy(col("category_count").desc())
# Apply window function
grouped_df_t7 = grouped_df_t7.withColumn("rank", rank().over(window_spec))
# Filter for most popular category for each age group
result_df_t7 = grouped_df_t7.filter(col("rank") == 1).drop("rank")

In [ ]:
display(result_df_t7)

In [ ]:
# Task 8: Find the median follower count for different age groups
# What is the median follower count for users in the following age groups:
#  18-24, 25-35, 36-50, +50
# Query should return DataFrame containing age_group, median_follower_count

from pyspark.sql.functions import when
# Join pin and user DataFrames
combined_df_t8 = cleaned_df_pin.join(cleaned_df_user, cleaned_df_pin["ind"] == cleaned_df_user["ind"], "inner")
# Categorise age into ranges
combined_df_t8 = combined_df_t8.withColumn("age_group",
                                           when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                           .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                           .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                           .otherwise("+50"))

# Calculate median for each age group
age_groups = ["18-24", "25-35", "36-50", "+50"]
medians = []
for group in age_groups:
    # Filter DataFrame for current age group in for loop
    temp_df = combined_df_t8.filter(col("age_group") == group)
    # Calculate median follower count for current age group
    median = temp_df.stat.approxQuantile("follower_count", [0.5], 0.01)
    # Store result
    medians.append((group, median[0]))

# Convert list of medians into DataFrame
median_df = spark.createDataFrame(medians, ["age_group", "median_follower_count"])

# # Select relevant columns
# combined_df_t8 = combined_df_t8.select("age_group", "follower_count")

# # Group by age_group and aggregate by median follower_count
# grouped_df_t8 = combined_df_t8.groupBy("age_group").agg(max("follower_count")-(min("follower_count"))).alias("median_follower_count")

In [ ]:
display(median_df)

In [ ]:
# Task 9: Find how many users have joined each year (2015-2020)?
# Query should return DataFrame that contains post_year and number_users_joined

# Join pin and user DataFrames
combined_df_t9 = cleaned_df_pin.join(cleaned_df_user, cleaned_df_pin["ind"] == cleaned_df_user["ind"], "inner")
# Select relevant columns
combined_df_t9 = combined_df_t9.select("date_joined")
# Extract year
task9_df = combined_df_t9.withColumn("post_year", year("date_joined"))
# Group and aggregate
task9_df = task9_df.groupBy("post_year").agg(count("date_joined").alias("number_users_joined"))

In [ ]:
display(task9_df)

In [ ]:
# Task 10: Find the median follower count of users based on their joining year (2015-2020)
#  Query should return DataFrame containing post_year and median_follower_count

from pyspark.sql.functions import year
from pyspark.sql.utils import AnalysisException

# Join pin and user DataFrames
combined_df_t10 = cleaned_df_pin.join(cleaned_df_user, cleaned_df_pin["ind"] == cleaned_df_user["ind"], "inner")
# Select relevant columns
combined_df_t10 = combined_df_t10.select("date_joined", "follower_count")
# Extract year
task10_df = combined_df_t10.withColumn("post_year", year("date_joined")).drop("date_joined")

# Calculate median follower count using approxQuantile
post_years = [2015, 2016, 2017, 2018, 2019, 2020]
medians = []
for year in post_years:
    try:
        # Filter DataFrame for the current posting year
        temp_df = task10_df.filter(col("post_year") == year)

        # Calculate median follower count for current posting year
        median = temp_df.stat.approxQuantile("follower_count", [0.5], 0.01)
        # Store the result
        medians.append((year, median[0]))
    # Append None if median is null
    except IndexError:
        medians.append((year, None))

median_df_t10 = spark.createDataFrame(medians, ["post_year", "median_follower_count"])

In [ ]:
display(median_df_t10)

In [ ]:
# Task 11: Find the median follower count of users based on their joining year and age group
# 2015-2020, [18-24, 25-35, 36-50, +50]
# Your query should return age_group, post_year, median_follower_count

from pyspark.sql.functions import year, col
# Join pin and user DataFrames
combined_df_t11 = (cleaned_df_pin.join(cleaned_df_user, cleaned_df_pin["ind"] == cleaned_df_user["ind"], "inner")
                                .join(cleaned_df_geo, cleaned_df_pin["ind"] == cleaned_df_geo["ind"], "inner"))
# Extract year from timestamp column
combined_df_t11 = combined_df_t11.withColumn("post_year", year("date_joined"))
# Create age_group column
task11_df = combined_df_t11.withColumn("age_group",
                                           when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                           .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                           .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                           .otherwise("+50"))
# Select relevant columns
task11_df = task11_df.select("age_group","post_year", "follower_count")

In [ ]:
# Calculate median_follower_count using approxQuantile
post_years = [2015, 2016, 2017, 2018, 2019, 2020]
age_groups = ["18-24", "25-35", "36-50", "+50"]
medians = []
for year in post_years:
    # Filter DataFrame for the current posting year
    year_df = task11_df.filter(col("post_year") == year)
    for age in age_groups:
        try:
            # Filter DataFrame for the current age group
            age_df = year_df.filter(col("age_group") == age)

            # Calculate median follower count for current age group
            median = age_df.stat.approxQuantile("follower_count", [0.5], 0.01)

            # Store the result
            medians.append((year, age, median[0]))
        except IndexError:
            medians.append((year, age, None))

median_df_t11 = spark.createDataFrame(medians, ["post_year", "age_group", "median_follower_count"])



In [ ]:
display(median_df_t11)

In [ ]:
display(cleaned_df_pin)
display(cleaned_df_geo)
display(cleaned_df_user)